<a href="https://colab.research.google.com/github/Alum-G/Alum-G/blob/main/Fashion_Mnist_Optimised_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install keras.tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 42.7 MB/s 
     |████████████████████████████████| 1.6 MB 14.4 MB/s 


In [19]:
import tensorflow as tf
from tensorflow import keras #For tf version 2.0 and above
import numpy as np

print(tf.__version__)

2.8.2


In [20]:
fashion_mnist = keras.datasets.fashion_mnist

In [21]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data() #Load data from fashion MNIST 

In [22]:
#Scale Images
train_images = train_images/255
test_images = test_images/255

In [23]:
train_images[1]


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.00392157, 0.        , 0.        , 0.        , 0.        ,
        0.16078431, 0.7372549 , 0.40392157, 0.21176471, 0.18823529,
        0.16862745, 0.34117647, 0.65882353, 0.52156863, 0.0627451 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.00392157, 0.        ,
        0.        , 0.        , 0.19215686, 0.53333333, 0.85882353,
        0.84705882, 0.89411765, 0.9254902 , 1.        , 1.        ,
        1.        , 1.        , 0.85098039, 0.84313725, 0.99607843,
        0.90588235, 0.62745098, 0.17647059, 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.05490196, 0.69019608, 0.87058824, 0.87843137, 0.83137255,
        0.79607843, 0.77647059, 0.76862745, 0.78431373, 0.84313725,
        0.8       , 0.7921

In [24]:
#Reshape Images
train_images = train_images.reshape(len(train_images),28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

HyperParameter Tuning

In [25]:
def build_model(hp):
  model = keras.Sequential([keras.layers.Conv2D(filters = hp.Int("conv_1_filter", min_value = 32, max_value = 256, step = 16),
                                                kernel_size = hp.Choice("conv_1_kernel", values = [3, 5]),
                                                activation = "relu",
                                                input_shape = (28, 28, 1)),
                            
                            keras.layers.Conv2D(filters = hp.Int("conv_2_filter", min_value = 32, max_value = 256, step = 16),
                                                kernel_size = hp.Choice("conv_2_kernel", values = [3,5]),
                                                activation = "relu"),
                            
                            keras.layers.Flatten(),

                            keras.layers.Dense(units = hp.Int("dense_1_units", min_value = 32, max_value = 256, step = 16),
                                               activation ="relu"),
                            
                            keras.layers.Dense(units = hp.Int("dense_1_units", min_value = 32, max_value = 256, step = 16),
                                               activation = "relu"),
                            
                            keras.layers.Dense(10, activation = "softmax") #Final Layer to generate output hence 10 neurons
                            ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2], [1e-3])),
                loss = "sparse_categorical_crossentropy",
                metrics = ["accuracy"])
  return model

In [26]:
import kerastuner
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [27]:
tuner_search = RandomSearch(build_model,
                            objective = "val_accuracy",
                            max_trials = 5,
                            directory = "output",
                            project_name = "Mnist Fashion Data")

In [28]:
tuner_search.search(train_images, train_labels,
                    epochs = 3,
                    validation_split = 0.1)

In [29]:
model = tuner_search.get_best_models(num_models=1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 192)       1920      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 240)       414960    
                                                                 
 flatten (Flatten)           (None, 138240)            0         
                                                                 
 dense (Dense)               (None, 64)                8847424   
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 9,269,114
Trainable params: 9,269,114
Non-

In [31]:
model.fit(train_images, train_labels, epochs = 10, validation_split = 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 21s 12ms/step - loss: 0.3179 - accuracy: 0.8861 - val_loss: 0.3513 - val_accuracy: 0.8752
Epoch 5/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.2999 - accuracy: 0.8916 - val_loss: 0.3652 - val_accuracy: 0.8782
Epoch 6/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.2849 - accuracy: 0.8975 - val_loss: 0.3661 - val_accuracy: 0.8783
Epoch 7/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.2693 - accuracy: 0.9046 - val_loss: 0.3957 - val_accuracy: 0.8795
Epoch 8/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.2597 - accuracy: 0.9078 - val_loss: 0.3840 - val_accuracy: 0.8823
Epoch 9/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.2589 - accuracy: 0.9094 - val_loss: 0.3785 - val_accuracy: 0.8830
Epoch 10/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.2408 - accuracy: 0.9160 - val_loss: 0.3921 